# Edamam

In [1]:
import requests
import collections
import pandas as pd

api_key = "7d29b3dffb6960d50759c3e8ab11da8d"

In [2]:
def get_recipe(dish):
    
    #call API
    url = f"https://api.edamam.com/api/recipes/v2?type=public&q={dish}&app_id=72116c1e&app_key={api_key}"

    response = requests.request("GET",url)
    recipe = response.json()["hits"][0]['recipe']
    
    return recipe

In [48]:
def get_ingredients(dish):
    
    #get ingredients from the call
    recipe = get_recipe(dish)
    servings = recipe['yield']
    ingredients = recipe['ingredients']
        
    new_dict = collections.defaultdict(list)
    print(new_dict)
    
    for i in ingredients:
        new_dict["ingredient"].append(i['food'].lower())
        new_dict["weight"].append(round((i['weight'] / servings), 2))
#         new_dict["metric"].append(i['measure'])
        
    return new_dict

In [49]:
output_dict = get_ingredients('hamburger')

output_df=pd.DataFrame(output_dict)

defaultdict(<class 'list'>, {})


In [50]:
output_df

,ingredient,weight
0,ground beef,56.70
1,salt,0.64
2,black pepper,0.32
3,hamburger bun,21.00
4,tomato,16.88
5,iceberg,2.50
6,onion,10.40


In [23]:
from os import name
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer 
import re

In [25]:
ingredient_file_path='../Emission_computing/final_ingredients_emissions.csv'
df=pd.read_csv(ingredient_file_path,error_bad_lines=False)

/Users/jackjohson/.pyenv/versions/lewagon/envs/FritzWebInterface/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 280: expected 2 fields, saw 4\n'


In [26]:
def match_ingredients(output_df):
    final_dict={'ingredient':[], 'emission':[],'weight':[]}
    missing_ingredients={'missing_ingredients':[]}
    
    for ingredient in output_df['ingredient']:
        
        # 1. Lemmatize 
        ingredient_words=re.split('\s+', ingredient)
        lemmatizer = WordNetLemmatizer()
        lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in ingredient_words])
        ingredient_words=re.split('\s+', lemmatized_output)
        
        # 2. Try the whole lemmatized sentence
        if df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].ingredient.values.size>0:
                final_dict['ingredient'].append(df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].ingredient.values[0])
                final_dict['emission'].append(df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].emissions.values[0])
                final_dict['weight'].append(output_df[output_df["ingredient"]==ingredient].weight.iloc[0])
                
#                 final_dict['value'].append(output_df[output_df["ingredient"]==ingredient].value.iloc[0])
#                 final_dict['metric'].append(output_df[output_df["ingredient"]==ingredient].metric.iloc[0])
        # False
        
        # 3. If the lemmatized output does not work, try the words
        else:
            for word in ingredient_words:
                # "ground" 
                # "beef"
                try:
                    final_dict['ingredient'].append(df[df['ingredient'].str.match(r'.*'+str(word)+'.*')== True].ingredient.values[0])
                    final_dict['emission'].append(df[df['ingredient'].str.match(r'.*'+str(word)+'.*')== True].emissions.values[0])
                    final_dict['weight'].append(output_df[output_df["ingredient"]==ingredient].weight.iloc[0])

#                     final_dict['value'].append(output_df[output_df["ingredient"]==ingredient].value.iloc[0])
#                     final_dict['metric'].append(output_df[output_df["ingredient"]==ingredient].metric.iloc[0])                    
                except IndexError:
                    missing_ingredients['missing_ingredients'].append(ingredient)
            
#     final_df=pd.DataFrame.from_dict(final_dict)
#     # Adding the columns "value" and metric from output_dict
#     final_df=final_df.merge(output_df,on="ingredient")
    final_df=pd.DataFrame(final_dict)

    return final_df,missing_ingredients
    


final_df, missing_ingredients=match_ingredients(output_df)

In [27]:
final_df

,ingredient,emission,weight
0,groundnut oil,900.0,56.6990
1,beef,43330.0,56.6990
2,salt,1133.0,0.6448
3,black pepper,1133.0,0.3224
4,hamburger bun,1215.0,21.0000
5,tomatoe,2905.0,16.8750
6,iceberg,2063.0,2.5000
7,onion,1051.0,10.4000


In [40]:
# 3. Convert the values
def convert(final_df):
#     output_df=final_df.merge(conv_df, on='metric')
    final_df.emission=final_df.emission.round()
    final_df["calculated gCO2e"]=final_df["emission"]*final_df["weight"]*0.001# *output_df["to_Kg_multiplier"]
    return final_df
# print('OK step 3')


In [41]:
result = convert(final_df)

In [44]:
emission = result['calculated gCO2e'].sum()
emission

2599.5173826000005

In [ ]:
for i in output['ingredient']:
    i.to_csv()

In [ ]:
for i in output['metric']:
    print(i)

In [ ]:
pd.DataFrame.from_dict(output)

## Metrics

In [ ]:
df_metrics = pd.read_csv("Emission_computing/raw_metrics.csv")

In [ ]:
df_metrics.shape

In [ ]:
df_metrics.duplicated().sum()

In [ ]:
df_metrics.drop_duplicates(inplace=True, ignore_index=True)

In [ ]:
df_metrics.shape

In [ ]:
df_metrics

In [ ]:
df_metrics.to_csv("Emission_computing/clean_metrics.csv", index = False)

## Ingredients

In [ ]:
df_ingredients = pd.read_csv("Emission_computing/raw_ingredients.csv")

In [ ]:
df_ingredients.shape

In [ ]:
df_ingredients.duplicated().sum()

In [ ]:
df_ingredients.drop_duplicates(inplace=True, ignore_index=True)

In [ ]:
df_ingredients.shape

In [ ]:
df_ingredients()

In [ ]:
df_ingredients.to_csv("Emission_computing/clean_ingredients.csv", index = False)

## Bunch of ingredients

In [52]:
raw_dishes = ['macarons',
 'french_toast',
 'lobster_bisque',
 'prime_rib',
 'pork_chop',
 'guacamole',
 'baby_back_ribs',
 'mussels',
 'beef_carpaccio',
 'poutine',
 'hot_and_sour_soup',
 'seaweed_salad',
 'foie_gras',
 'dumplings',
 'peking_duck',
 'takoyaki',
 'bibimbap',
 'falafel',
 'pulled_pork_sandwich',
 'lobster_roll_sandwich',
 'carrot_cake',
 'beet_salad',
 'panna_cotta',
 'donuts',
 'red_velvet_cake',
 'grilled_cheese_sandwich',
 'cannoli',
 'spring_rolls',
 'shrimp_and_grits',
 'clam_chowder',
 'omelette',
 'fried_calamari',
 'caprese_salad',
 'oysters',
 'scallops',
 'ramen',
 'grilled_salmon',
 'croque_madame',
 'filet_mignon',
 'hamburger',
 'spaghetti_carbonara',
 'miso_soup',
 'bread_pudding',
 'lasagna',
 'crab_cakes',
 'cheesecake',
 'spaghetti_bolognese',
 'cup_cakes',
 'creme_brulee',
 'waffles',
 'fish_and_chips',
 'paella',
 'macaroni_and_cheese',
 'chocolate_mousse',
 'ravioli',
 'chicken_curry',
 'caesar_salad',
 'nachos',
 'tiramisu',
 'frozen_yogurt',
 'ice_cream',
 'risotto',
 'club_sandwich',
 'strawberry_shortcake',
 'steak',
 'churros',
 'garlic_bread',
 'baklava',
 'bruschetta',
 'hummus',
 'chicken_wings',
 'greek_salad',
 'tuna_tartare',
 'chocolate_cake',
 'gyoza',
 'eggs_benedict',
 'deviled_eggs',
 'samosa',
 'sushi',
 'breakfast_burrito',
 'ceviche',
 'beef_tartare',
 'apple_pie',
 'huevos_rancheros',
 'beignets',
 'pizza',
 'edamame',
 'french_onion_soup',
 'hot_dog',
 'tacos',
 'chicken_quesadilla',
 'pho',
 'gnocchi',
 'pancakes',
 'fried_rice',
 'cheese_plate',
 'onion_rings',
 'escargots',
 'sashimi',
 'pad_thai',
 'french_fries']

In [225]:
just_few = raw_dishes[:10]

In [226]:
just_few

['macarons',
 'french_toast',
 'lobster_bisque',
 'prime_rib',
 'pork_chop',
 'guacamole',
 'baby_back_ribs',
 'mussels',
 'beef_carpaccio',
 'poutine']

In [227]:
dishes = []
for i in just_few:
    dishes.append(i.replace("_", " "))

In [228]:
dishes

['macarons',
 'french toast',
 'lobster bisque',
 'prime rib',
 'pork chop',
 'guacamole',
 'baby back ribs',
 'mussels',
 'beef carpaccio',
 'poutine']

In [229]:
output = []
for dish in dishes:
    output.append(get_recipe(dish))
    print(output)

[{'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_c5e8f50f8455601e148e548f3a4af864', 'label': 'Caramel Banana Macarons', 'image': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'images': {'THUMBNAIL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-s.jpg', 'width': 100, 'height': 100}, 'SMALL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-m.jpg', 'width': 200, 'height': 200}, 'REGULAR': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'width': 300, 'height': 300}, 'LARGE': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-l.jpg', 'width': 600, 'height': 600}}, 'source': 'Tartelette', 'url': 'http://www.tarteletteblog.com/2009/10/recipes-macarons-it-takes-all-sorts-of.html', 'shareAs': 'http://www.edamam.com/recipe/caramel-banana-macarons-c5e8f50f8455601e148e548f3a4af864/macarons', 'yield': 6.0, 'dietLabels': ['Low-Carb', 'Low-S

[{'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_c5e8f50f8455601e148e548f3a4af864', 'label': 'Caramel Banana Macarons', 'image': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'images': {'THUMBNAIL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-s.jpg', 'width': 100, 'height': 100}, 'SMALL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-m.jpg', 'width': 200, 'height': 200}, 'REGULAR': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'width': 300, 'height': 300}, 'LARGE': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-l.jpg', 'width': 600, 'height': 600}}, 'source': 'Tartelette', 'url': 'http://www.tarteletteblog.com/2009/10/recipes-macarons-it-takes-all-sorts-of.html', 'shareAs': 'http://www.edamam.com/recipe/caramel-banana-macarons-c5e8f50f8455601e148e548f3a4af864/macarons', 'yield': 6.0, 'dietLabels': ['Low-Carb', 'Low-S

[{'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_c5e8f50f8455601e148e548f3a4af864', 'label': 'Caramel Banana Macarons', 'image': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'images': {'THUMBNAIL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-s.jpg', 'width': 100, 'height': 100}, 'SMALL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-m.jpg', 'width': 200, 'height': 200}, 'REGULAR': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'width': 300, 'height': 300}, 'LARGE': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-l.jpg', 'width': 600, 'height': 600}}, 'source': 'Tartelette', 'url': 'http://www.tarteletteblog.com/2009/10/recipes-macarons-it-takes-all-sorts-of.html', 'shareAs': 'http://www.edamam.com/recipe/caramel-banana-macarons-c5e8f50f8455601e148e548f3a4af864/macarons', 'yield': 6.0, 'dietLabels': ['Low-Carb', 'Low-S

[{'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_c5e8f50f8455601e148e548f3a4af864', 'label': 'Caramel Banana Macarons', 'image': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'images': {'THUMBNAIL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-s.jpg', 'width': 100, 'height': 100}, 'SMALL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-m.jpg', 'width': 200, 'height': 200}, 'REGULAR': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'width': 300, 'height': 300}, 'LARGE': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-l.jpg', 'width': 600, 'height': 600}}, 'source': 'Tartelette', 'url': 'http://www.tarteletteblog.com/2009/10/recipes-macarons-it-takes-all-sorts-of.html', 'shareAs': 'http://www.edamam.com/recipe/caramel-banana-macarons-c5e8f50f8455601e148e548f3a4af864/macarons', 'yield': 6.0, 'dietLabels': ['Low-Carb', 'Low-S

[{'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_c5e8f50f8455601e148e548f3a4af864', 'label': 'Caramel Banana Macarons', 'image': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'images': {'THUMBNAIL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-s.jpg', 'width': 100, 'height': 100}, 'SMALL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-m.jpg', 'width': 200, 'height': 200}, 'REGULAR': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'width': 300, 'height': 300}, 'LARGE': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-l.jpg', 'width': 600, 'height': 600}}, 'source': 'Tartelette', 'url': 'http://www.tarteletteblog.com/2009/10/recipes-macarons-it-takes-all-sorts-of.html', 'shareAs': 'http://www.edamam.com/recipe/caramel-banana-macarons-c5e8f50f8455601e148e548f3a4af864/macarons', 'yield': 6.0, 'dietLabels': ['Low-Carb', 'Low-S

[{'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_c5e8f50f8455601e148e548f3a4af864', 'label': 'Caramel Banana Macarons', 'image': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'images': {'THUMBNAIL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-s.jpg', 'width': 100, 'height': 100}, 'SMALL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-m.jpg', 'width': 200, 'height': 200}, 'REGULAR': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'width': 300, 'height': 300}, 'LARGE': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-l.jpg', 'width': 600, 'height': 600}}, 'source': 'Tartelette', 'url': 'http://www.tarteletteblog.com/2009/10/recipes-macarons-it-takes-all-sorts-of.html', 'shareAs': 'http://www.edamam.com/recipe/caramel-banana-macarons-c5e8f50f8455601e148e548f3a4af864/macarons', 'yield': 6.0, 'dietLabels': ['Low-Carb', 'Low-S

[{'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_c5e8f50f8455601e148e548f3a4af864', 'label': 'Caramel Banana Macarons', 'image': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'images': {'THUMBNAIL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-s.jpg', 'width': 100, 'height': 100}, 'SMALL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-m.jpg', 'width': 200, 'height': 200}, 'REGULAR': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'width': 300, 'height': 300}, 'LARGE': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-l.jpg', 'width': 600, 'height': 600}}, 'source': 'Tartelette', 'url': 'http://www.tarteletteblog.com/2009/10/recipes-macarons-it-takes-all-sorts-of.html', 'shareAs': 'http://www.edamam.com/recipe/caramel-banana-macarons-c5e8f50f8455601e148e548f3a4af864/macarons', 'yield': 6.0, 'dietLabels': ['Low-Carb', 'Low-S

[{'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_c5e8f50f8455601e148e548f3a4af864', 'label': 'Caramel Banana Macarons', 'image': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'images': {'THUMBNAIL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-s.jpg', 'width': 100, 'height': 100}, 'SMALL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-m.jpg', 'width': 200, 'height': 200}, 'REGULAR': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'width': 300, 'height': 300}, 'LARGE': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-l.jpg', 'width': 600, 'height': 600}}, 'source': 'Tartelette', 'url': 'http://www.tarteletteblog.com/2009/10/recipes-macarons-it-takes-all-sorts-of.html', 'shareAs': 'http://www.edamam.com/recipe/caramel-banana-macarons-c5e8f50f8455601e148e548f3a4af864/macarons', 'yield': 6.0, 'dietLabels': ['Low-Carb', 'Low-S

[{'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_c5e8f50f8455601e148e548f3a4af864', 'label': 'Caramel Banana Macarons', 'image': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'images': {'THUMBNAIL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-s.jpg', 'width': 100, 'height': 100}, 'SMALL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-m.jpg', 'width': 200, 'height': 200}, 'REGULAR': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'width': 300, 'height': 300}, 'LARGE': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-l.jpg', 'width': 600, 'height': 600}}, 'source': 'Tartelette', 'url': 'http://www.tarteletteblog.com/2009/10/recipes-macarons-it-takes-all-sorts-of.html', 'shareAs': 'http://www.edamam.com/recipe/caramel-banana-macarons-c5e8f50f8455601e148e548f3a4af864/macarons', 'yield': 6.0, 'dietLabels': ['Low-Carb', 'Low-S

[{'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_c5e8f50f8455601e148e548f3a4af864', 'label': 'Caramel Banana Macarons', 'image': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'images': {'THUMBNAIL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-s.jpg', 'width': 100, 'height': 100}, 'SMALL': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-m.jpg', 'width': 200, 'height': 200}, 'REGULAR': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3.jpg', 'width': 300, 'height': 300}, 'LARGE': {'url': 'https://www.edamam.com/web-img/213/213b61a70b6daae1ad62c46c775295a3-l.jpg', 'width': 600, 'height': 600}}, 'source': 'Tartelette', 'url': 'http://www.tarteletteblog.com/2009/10/recipes-macarons-it-takes-all-sorts-of.html', 'shareAs': 'http://www.edamam.com/recipe/caramel-banana-macarons-c5e8f50f8455601e148e548f3a4af864/macarons', 'yield': 6.0, 'dietLabels': ['Low-Carb', 'Low-S

In [231]:
print(output[1])

{'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_47ccfc27c1c4e82dff4a3228345a75af', 'label': 'Villa 525 French Toast', 'image': 'https://www.edamam.com/web-img/45d/45d55dcf4440339586aa471a25e66b78.jpg', 'images': {'THUMBNAIL': {'url': 'https://www.edamam.com/web-img/45d/45d55dcf4440339586aa471a25e66b78-s.jpg', 'width': 100, 'height': 100}, 'SMALL': {'url': 'https://www.edamam.com/web-img/45d/45d55dcf4440339586aa471a25e66b78-m.jpg', 'width': 200, 'height': 200}, 'REGULAR': {'url': 'https://www.edamam.com/web-img/45d/45d55dcf4440339586aa471a25e66b78.jpg', 'width': 300, 'height': 300}, 'LARGE': {'url': 'https://www.edamam.com/web-img/45d/45d55dcf4440339586aa471a25e66b78-l.jpg', 'width': 600, 'height': 600}}, 'source': 'Food52', 'url': 'http://www.food52.com/recipes/4604_villa_525_french_toast', 'shareAs': 'http://www.edamam.com/recipe/villa-525-french-toast-47ccfc27c1c4e82dff4a3228345a75af/french+toast', 'yield': 6.0, 'dietLabels': [], 'healthLabels': ['Vegetarian', 'Pescataria

In [160]:
ingredients = []
comiditas = []

for _ in range(len(output)):
    ingredients.append(output[_]['ingredients'])

for _ in range(len(ingredients)):
    for i in range(len(ingredients[_])):
        comiditas.append([ingredients[_][i]['food'], ingredients[_][i]['measure'], \
                         ingredients[_][i]['quantity']])     
        
comiditas

[['Flour tortillas', '<unit>', 4.0],
 ['chicken meat', 'cup', 1.0],
 ['jack cheese', 'pound', 0.25],
 ['apple', '<unit>', 1.0],
 ['salsa', 'cup', 0.25],
 ['onions', '<unit>', 2.0],
 ['ginger', '<unit>', 1.0],
 ['beef shin', 'pound', 3.0],
 ['oxtail', 'pound', 2.0],
 ['beef chuck', 'pound', 1.0],
 ['beef brisket', 'pound', 1.0],
 ['star anise', 'pod', 3.0],
 ['cinnamon stick', '<unit>', 1.0],
 ['fennel seeds', 'teaspoon', 1.0],
 ['cloves', '<unit>', 4.0],
 ['coriander seeds', 'teaspoon', 1.0],
 ['fish sauce', 'cup', 0.25],
 ['sugar', 'tablespoon', 2.0],
 ['Kosher salt', None, 0.0],
 ['noodles', 'serving', 7.0],
 ['beef flank steak', 'pound', 1.0],
 ['herbs', 'cup', 2.5],
 ['bean sprouts', 'cup', 2.5],
 ['scallions', 'cup', 0.5],
 ['chilis', None, 0.0],
 ['limes', '<unit>', 2.0],
 ['Hoisin sauce', None, 0.0],
 ['unsalted butter', '<unit>', 2.0],
 ['Gnocchi', '<unit>', 1.0],
 ['Parmesan cheese', 'cup', 0.5],
 ['parsley', 'tablespoon', 1.0],
 ['chives', 'tablespoon', 1.0],
 ['all-purpose f

In [161]:
comiditas_final += comiditas

In [214]:
units=[]
for item in comiditas_final:
    if item[1] == '<unit>':
        units.append(item)

In [221]:
units_df = pd.DataFrame(units)
units_df

,0,1,2
0,coriander,<unit>,3.0
1,lime leaf,<unit>,3.0
2,prawn,<unit>,6.0
3,green chilli,<unit>,6.0
4,scallion,<unit>,1.0
...,...,...,...
137,unsalted butter,<unit>,2.0
138,Gnocchi,<unit>,1.0
139,eggs,<unit>,2.0
140,eggs,<unit>,4.0


In [224]:
for item in units_df[0].drop_duplicates():
    print(item)

coriander
lime leaf
prawn
green chilli
scallion
green chile
egg white
dumpling wrappers
scallions
granny smith
Egg whites
eggs
bananas
lime
olive
standing rib roast
egg yolk
tomatoes
Russet potatoes
apricots
donuts
cannoli shells
pineapple
spring roll wrappers
green onions
jalapeño
andouille sausage
bay leaf
oysters
lemons
cooked chicken breast
egg
shallot
cherry tomatoes
hamburger bun
Potato Chips
shallots
whole-grain waffles
russet potatoes
onion
chicken drumettes
lemon
cheese ravioli
onions
whole tomatoes
white onion
poblano pepper
cheese
Egg Yolks
sponge Fingers
Strawberry
baguette
plum tomatoes
dill pickle
whole wheat tortillas
carrot
persian cucumber
red bell pepper
Kalamata olives
cucumber
red onion
avocado
Chocolate Cake
firm tofu
dried shiitake mushrooms
gyoza wrappers
english muffins
butter
hard boiled eggs
tomato
potatoes
limes
chili
sweet potato
anchovy fillets
double pie crust
corn tortilla
yellow onions
wonton wrappers
red onions
yellow onion
green pepper
Corn Tortillas
F

In [162]:
len(comiditas_final)

956

In [205]:
all_comiditas = []
for ingridient in comiditas_final:
    all_comiditas.append(ingridient.replace("-", " ").lower())

In [206]:
len(all_comiditas)

893

In [207]:
comiditas_final_dict = {"ingredient": all_comiditas}

In [208]:
df = pd.DataFrame(comiditas_final_dict)

In [209]:
df.drop_duplicates(inplace=True)

In [210]:
df.duplicated().sum()

0

In [211]:
df

,ingredient
0,kosher salt
1,lobster tails
2,extra virgin olive oil
3,fish stock
4,sauvignon blanc
...,...
880,rice noodles
881,agave
883,shoyu
888,tofu


In [212]:
df.to_csv("ALL_ingredients.csv", index=False)